In [187]:

import os
import pickle
import pandas as pd
import sys
import sys
from memory_profiler import profile
import pandas as pd
import numpy as np
sys.path.append('/home/ak/Documents/Research/PaperCode/MultiKernelLearning')
sys.path.append('/home/ak/Documents/Research/PaperCode/singlekernelclf/')
import jsonpickle
from collections import defaultdict
from sklearn.svm import SVC
from collections import OrderedDict
import os
from sklearn.metrics import accuracy_score
import torch


from fileutils import new_feature_utils as nfu
from fileutils.new_feature_utils import CreateMarketFeatures
import multiprocessing
import mkl_data_processing as mkldp
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
import matplotlib as mpl
import matplotlib.pyplot as plt
styleFiles = os.listdir(mpl.get_configdir())
styleFileIdx = 0

plt.style.use(os.path.join(mpl.get_configdir(), styleFiles[styleFileIdx]))

In [188]:
def open_pickle_filepath(pickle_file):
    pickle_to_file = pickle.load(open(pickle_file, "rb"), encoding='latin1')

    return pickle_to_file

In [189]:
files_location = '/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSPredictions/SimpleMKL/'
symbols = sorted(os.listdir(files_location))
#processed results path
processed_location = '/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSProcessed'

In [695]:
symbolIDX =  12 #pick a symbol
label_idx = 6 #pick a label
label_string= 'Label_idx:_'+str(label_idx)
print('you are doing symbol', symbols[symbolIDX], 'and label', label_string)
# now we take out the files for the symbol 
symbol_path = os.path.join(files_location,str(sorted(symbols)[symbolIDX]))
symbolFiles = os.listdir(os.path.join(files_location,str(sorted(symbols)[symbolIDX])))
# now isolate all the label+ symbol combos
symbolLabelFiles = [f for f in symbolFiles if str(label_string) in f]
path_to_store = os.path.join(processed_location, symbols[symbolIDX])
number_of_files = len(symbolLabelFiles) 

# this is the number of files that correspond to the label/symbol combo

print('total files for the symbol:', len(symbolFiles))
print('and for this label')
print(number_of_files)

you are doing symbol KGF.L and label Label_idx:_6
total files for the symbol: 1836
and for this label
121


In [696]:
path_to_store

'/media/ak/DataOnly/ExperimentCommonLocs/MKLOOSProcessed/KGF.L'

In [697]:
dates_chunks_keys = dict()
chunks_list =list()
keys_list = list()

for file_to_load_idx in range(0,number_of_files):
    # go through all of the files that are a combination of symbols and labels
    file_to_load_path = os.path.join(os.path.join(files_location, str(symbols[symbolIDX])), symbolLabelFiles[file_to_load_idx])
    if os.path.getsize(file_to_load_path)> 0:
        # extract the key - these are the symbol feature path keys in the  main code
        file_keys = list(open_pickle_filepath(file_to_load_path).keys()) # keys
        keys_list.append(file_keys[0])
        dates_chunks_keys[file_to_load_idx] = file_keys
        chunks_list.append(open_pickle_filepath(file_to_load_path)[file_keys[0]])
    else:
        continue

        

In [698]:
dates_df = pd.DataFrame((np.unique(list(dates_chunks_keys.values()))))
list_of_dfs_across_days = list()
for chunk_list_item in range(0,len(chunks_list)-1):
    chunk_list_item_dict = chunks_list[chunk_list_item]
    chunk_list_item_keys = list(chunk_list_item_dict.keys())
    list_of_dicts=list()
    for key in chunk_list_item_keys:
        list_of_dicts.append(pd.DataFrame(chunk_list_item_dict[key]))
    list_of_dfs_across_days.append(pd.concat(list_of_dicts,axis=1).median(axis=1))


In [699]:
list_of_dfs_across_days

[Hamming Loss    0.54
 accuracy        0.46
 f1- macro       0.22
 f1- micro       0.46
 f1- weighted    0.32
 precision       0.30
 recall          0.46
 dtype: float64, Hamming Loss    0.525
 accuracy        0.475
 f1- macro       0.220
 f1- micro       0.475
 f1- weighted    0.330
 precision       0.310
 recall          0.475
 dtype: float64, Hamming Loss    0.54
 accuracy        0.46
 f1- macro       0.22
 f1- micro       0.46
 f1- weighted    0.32
 precision       0.30
 recall          0.46
 dtype: float64, Hamming Loss    0.535
 accuracy        0.465
 f1- macro       0.220
 f1- micro       0.465
 f1- weighted    0.320
 precision       0.280
 recall          0.465
 dtype: float64, Hamming Loss    0.54
 accuracy        0.46
 f1- macro       0.22
 f1- micro       0.46
 f1- weighted    0.32
 precision       0.30
 recall          0.46
 dtype: float64, Hamming Loss    0.54
 accuracy        0.46
 f1- macro       0.22
 f1- micro       0.46
 f1- weighted    0.32
 precision       0.30
 rec

In [710]:
df =pd.concat(list_of_dfs_across_days, axis =1)
df.median(axis=1)

Hamming Loss    0.54
accuracy        0.46
f1- macro       0.22
f1- micro       0.46
f1- weighted    0.32
precision       0.30
recall          0.46
dtype: float64

In [711]:
#df.to_csv('file_name.csv
location_for_df = os.path.join(path_to_store, symbols[symbolIDX] +"_Label_"+str(label_idx)+"_all_chunks.csv")
df.to_csv(location_for_df, index= True)
location_for_dates_df = os.path.join(path_to_store, symbols[symbolIDX] +"_Label_"+str(label_idx)+"_dates.csv")
dates_df.to_csv(location_for_dates_df, index = True)

In [712]:
df_test = pd.read_csv(location_for_df, index_col=0)
df_test.median(axis=1)

Hamming Loss    0.54
accuracy        0.46
f1- macro       0.22
f1- micro       0.46
f1- weighted    0.32
precision       0.30
recall          0.46
dtype: float64

# to do:
1. write a loop to go through all the labels for each symbol and store them in a seperate file
2. concentrate only on the symbols we know we have labels
3. pull the file from Single Kernel and compare them.
4. re-do the labels and the plots for the symbols that you are picking and describe them

In [703]:

# Path
def create_path(parent_dir, symbol):
    path = os.path.join(parent_dir, symbol)
    if os.path.isdir(path)==False:
        # Create the directory
    
        os.mkdir(path)
        print("Directory '%s' created" %path)
    else:
        print("Directory Exist")
        pass

In [704]:
# for symbolIDX in range(0, len(symbols)):
#     create_path(processed_location, symbols[symbolIDX])

In [705]:
symbols_to_load = os.listdir(processed_location)

In [706]:
list_files = list()
string = str('_Label_2')
for idx in range(0,40):
    files =os.listdir(os.path.join(processed_location, symbols_to_load[idx]))
    list_files.append([f for f in files if string in f])

        

In [707]:
list_files

[['LGEN.L_Label_2_all_chunks.csv', 'LGEN.L_Label_2_dates.csv'],
 [],
 [],
 ['BLT.L_Label_2_all_chunks.csv', 'BLT.L_Label_2_dates.csv'],
 ['KGF.L_Label_2_all_chunks.csv', 'KGF.L_Label_2_dates.csv'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['CEY.L_Label_2_all_chunks.csv', 'CEY.L_Label_2_dates.csv'],
 [],
 [],
 ['ITV.L_Label_2_all_chunks.csv', 'ITV.L_Label_2_dates.csv'],
 [],
 [],
 ['APF.L_Label_2_all_chunks.csv', 'APF.L_Label_2_dates.csv'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['CPG.L_Label_2_dates.csv', 'CPG.L_Label_2_all_chunks.csv'],
 ['BARC.L_Label_2_dates.csv', 'BARC.L_Label_2_all_chunks.csv'],
 [],
 [],
 [],
 ['MAB.L_Label_2_dates.csv', 'MAB.L_Label_2_all_chunks.csv'],
 [],
 ['LAND.L_Label_2_dates.csv', 'LAND.L_Label_2_all_chunks.csv']]

In [708]:
label_1_files = ['BARC.L_Label_1_all_chunks.csv',
    'APF.L_Label_1_all_chunks.csv', 
                 'CCL.L_Label_1_all_chunks.csv',
                 'AV.L_Label_1_all_chunks.csv',
                'BLT.L_Label_1_all_chunks.csv',
                'AZN.L_Label_1_all_chunks.csv',
                'KGF.L_Label_1_all_chunks.csv']

label_5_files = ['PSON.L_Label_5_all_chunks.csv', 
                 'PRU.L_Label_5_all_chunks.csv',
                 'AV.L_Label_5_all_chunks.csv', 
                 'APF.L_Label_5_all_chunks.csv', 
                 'MKS.L_Label_5_all_chunks.csv']

label_4_files =  ['NG.L_Label_4_all_chunks.csv',
  'BLT.L_Label_4_all_chunks.csv',
  'KGF.L_Label_4_all_chunks.csv', 
  'PRU.L_Label_4_all_chunks.csv',
  'RTO.L_Label_4_all_chunks.csv',
 'RR.L_Label_4_all_chunks.csv',
 'RSA.L_Label_4_all_chunks.csv',
'CEY.L_Label_4_all_chunks.csv',
'RB.L_Label_4_all_chunks.csv',
'ITV.L_Label_4_all_chunks.csv',
'PSON.L_Label_4_all_chunks.csv',
  'SDR.L_Label_4_all_chunks.csv',
  'APF.L_Label_4_all_chunks.csv',
'REL.L_Label_4_all_chunks.csv', 
'AZN.L_Label_4_all_chunks.csv',
'CCL.L_Label_4_all_chunks.csv',
 'SGE.L_Label_4_all_chunks.csv', 
 'RDSb.L_Label_4_all_chunks.csv', 
 'CPG.L_Label_4_all_chunks.csv',
'AV.L_Label_4_all_chunks.csv',
'MAB.L_Label_4_all_chunks.csv',
  'LAND.L_Label_4_all_chunks.csv']

label_2_files = ['BLT.L_Label_2_all_chunks.csv', 
                 'KGF.L_Label_2_all_chunks.csv',
                'CEY.L_Label_2_all_chunks.csv',
                 'CPG.L_Label_2_all_chunks.csv',
                 'BARC.L_Label_2_all_chunks.csv',
                 'ITV.L_Label_2_all_chunks.csv',
                 'APF.L_Label_2_all_chunks.csv' ,
                'LAND.L_Label_2_all_chunks.csv',
                'LGEN.L_Label_2_all_chunks.csv',
                 'MAB.L_Label_2_all_chunks.csv'
                ]


label_3_files =['LGEN.L_Label_3_all_chunks.csv',
                'APF.L_Label_3_all_chunks.csv',
                'AV.L_Label_3_all_chunks.csv',
                'MAB.L_Label_3_all_chunks.csv',
                'LAND.L_Label_3_all_chunks.csv',
               'LLOY.L_Label_3_all_chunks.csv',
               'RBS.L_Label_3_all_chunks.csv']

label_6_files = ['RSA.L_Label_6_all_chunks.csv',
                'WPP.L_Label_6_all_chunks.csv',
                'MKS.L_Label_6_all_chunks.csv',
                'RR.L_Label_6_all_chunks.csv',
                'LAND.L_Label_6_all_chunks.csv',
                'CEY.L_Label_6_all_chunks.csv']


In [709]:
row_labels = ['Hamming Loss',
 'accuracy',
 'f1- macro',
 'f1- micro',
 'f1- weighted',
 'precision',
 'recall']


label_dfs = {'1':label_1_dfs,
             '2':label_2_dfs,
             '3':label_3_dfs,
             '4':label_4_dfs,
             '5':label_5_dfs,
             '6':label_6_dfs
            }

label_dicts = {'1':label_1_dicts,
             '2':label_2_dicts,
             '3':label_3_dicts,
             '4':label_4_dicts,
             '5':label_5_dicts,
             '6':label_6_dicts
            }

label_files ={'1':label_1_files,
              '2':label_2_files,
              '3':label_3_files,
              '4':label_4_files,
              '5':label_5_files,
              '6':label_6_files
             }

In [637]:
labelIdx = str(6)
label_Idx_files = label_files[labelIdx]
label_Idx_dfs = label_dfs[labelIdx]
label_Idx_dicts = label_dicts[labelIdx]



for idx in range(0, len(label_Idx_files)):
    symbol = label_Idx_files[idx].split("_")[0]
    
    file_to_load = \
    pd.read_csv(os.path.join(processed_location, symbol, label_Idx_files[idx])).median(axis=1)
    label_Idx_dfs.append(file_to_load)
    file_to_load.index = row_labels
    label_Idx_dicts[symbol] = file_to_load

In [638]:
labels_dicts_processed = {'One': pd.DataFrame.from_dict(label_dicts['1']),
               'Two':  pd.DataFrame.from_dict(label_dicts['2']),
               'Three':  pd.DataFrame.from_dict(label_dicts['3']),
               'Four':  pd.DataFrame.from_dict(label_dicts['4']),
               'Five':  pd.DataFrame.from_dict(label_dicts['5']),
               'Six':  pd.DataFrame.from_dict(label_dicts['6']),}

In [676]:
symbols_for_label_one = list(labels_dicts_processed['One'].columns.values)
symbols_for_label_two = list(labels_dicts_processed['Two'].columns.values)
symbols_for_label_three = list(labels_dicts_processed['Three'].columns.values)
symbols_for_label_four = list(labels_dicts_processed['Four'].columns.values)
symbols_for_label_five = list(labels_dicts_processed['Five'].columns.values)
symbols_for_label_six = list(labels_dicts_processed['Six'].columns.values)

common_symbols = set(symbols_for_label_one)  - set(symbols_for_label_five) - \
set(symbols_for_label_four)


In [677]:
print(list(common_symbols))

['BARC.L']


In [687]:
df = pd.DataFrame()
for key in list(labels_dicts_processed.keys()):
    df[key] =labels_dicts_processed[str(key)].median(axis =1)

In [689]:
print(df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &   One &    Two &  Three &  Four &   Five &    Six \\
\midrule
Hamming Loss &  0.02 &  0.115 &   0.62 &  0.16 &  0.525 &  0.535 \\
accuracy     &  0.98 &  0.885 &   0.38 &  0.84 &  0.475 &  0.465 \\
f1- macro    &  0.49 &  0.470 &   0.26 &  0.46 &  0.240 &  0.220 \\
f1- micro    &  0.98 &  0.885 &   0.38 &  0.84 &  0.475 &  0.465 \\
f1- weighted &  0.97 &  0.840 &   0.31 &  0.77 &  0.330 &  0.320 \\
precision    &  0.96 &  0.790 &   0.30 &  0.71 &  0.290 &  0.265 \\
recall       &  0.98 &  0.885 &   0.38 &  0.84 &  0.475 &  0.465 \\
\bottomrule
\end{tabular}

